# **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


In [93]:
os.environ["CUDA_LAUNCH_BLOCKING"]='1'

In [1]:
import os
import random
import pandas as pd
import torch
import gc
gc.collect()
# os.environ["CUDA_VISIBLE_DEVICES"]='1'
torch.cuda.empty_cache()
# torch.cuda.clear_memory_allocated()
# torch.cuda.memory_summary(device=None, abbreviated=False)
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

!nvidia-smi

Sat Mar 12 01:19:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   59C    P0    42W / 180W |    353MiB /  8192MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

## Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw2/data) and upload it to the workspace**


### Download train/test metadata

In [2]:
# !pip install --upgrade gdown

# # Main link
# !gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# # Backup link 1
# # !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# # Bqckup link 2
# # !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

# !unzip -q libriphone.zip
# # !ls libriphone

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [3]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm
import numpy as np

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    
    tt_label = []
    tt_feat = []
    
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]
        #   tt_label.append(torch.tensor(label_dict[line[0]])) #############

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    # max_len = 3000000
    # X = torch.empty(max_len, 39 * concat_nframes)
    # if mode != 'test':
    #   y = torch.empty(max_len, dtype=torch.long)

    total_len = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        #print(cur_len) #####
        # feat = concat_feat(feat, concat_nframes)
        
        tt_feat.append(feat)
        total_len += cur_len
        
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname])
          tt_label.append(label)#torch.tensor(label_dict[line[0]]))
          

        # X[idx: idx + cur_len, :] = feat
        # if mode != 'test':
        #   y[idx: idx + cur_len] = label

        

    # X = X[:idx, :]
    # if mode != 'test':
    #   y = y[:idx]

    print(f'[INFO] {split} set')
    # print(X.shape)
    if mode != 'test':
    #   print(y.shape)
        print(np.array(tt_feat).shape,np.array(tt_label).shape)
        return tt_feat,tt_label,total_len
    #   return X, y, tt_feat, tt_label
    else:
        return tt_feat,total_len
    #   return X, tt_feat, tt_label


## Define Dataset

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
device

'cuda:0'

In [5]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = y ###############
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


## Define Model

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, hidden_layers, isBidirectional, output_dim):
        super(Classifier, self).__init__()

        D = 2 if isBidirectional else 1

        self.lstm = nn.GRU(
            input_size = input_dim,
            hidden_size = hidden_dim,
            num_layers = hidden_layers,
            batch_first = True,
            bidirectional = isBidirectional,
            dropout=0.2
        )
        self.fc = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(hidden_dim * D, output_dim)
        )

    def forward(self, x):
        output, _ = self.lstm(x)
        unpacked_out = output
        if type(output) == torch.nn.utils.rnn.PackedSequence:
            # unpacked
            unpacked_out, _ = pad_packed_sequence(output, batch_first=True)
        x = self.fc(unpacked_out)
        return x


## Hyper-parameters

In [7]:
# data prarameters
concat_nframes = 1              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.9               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 0                        # random seed
batch_size = 100                # batch size
num_epoch = 250                 # the number of training epoch
learning_rate = 0.001           # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 #* concat_nframes # the input dim of the model, you should not change the value
hidden_layer = 15               # the number of hidden layers
hidden_dim = 35                # the hidden dim

## Prepare dataset and model

In [8]:
import gc

# preprocess data
train_tt_feat, train_tt_label, train_total_len = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_tt_feat, val_tt_label, val_total_len = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

1it [00:00,  7.08it/s]

[Dataset] - # phone classes: 41, number of utterances for train: 3857


3857it [00:48, 79.28it/s] 
/tmp/ipykernel_3275/1127721884.py:105: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  print(np.array(tt_feat).shape,np.array(tt_label).shape)
/tmp/ipykernel_3275/1127721884.py:105: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(tt_feat).shape,np.array(tt_label).shape)


[INFO] train set
(3857,) (3857,)


8it [00:00, 67.43it/s]

[Dataset] - # phone classes: 41, number of utterances for val: 429


429it [00:06, 66.21it/s] 

[INFO] val set
(429,) (429,)


In [9]:
(train_tt_feat[0][0][0])

tensor(-1.2992)

In [10]:
train_set = LibriDataset(train_tt_feat, train_tt_label)
val_set = LibriDataset(val_tt_feat, val_tt_label)

In [11]:
train_set[0][0][0]

tensor([-1.2992, -0.5816, -0.3350,  0.2358,  0.7594,  0.0168, -0.4481, -0.3293,
         1.0392,  0.2015, -1.4177,  1.5647,  1.6671,  0.7037,  1.0508,  1.3092,
         0.6659, -0.1440, -0.7609,  0.0406,  0.7632, -0.3623,  0.4380,  0.8197,
         1.7101,  1.7190,  0.7870,  0.1252, -0.5424, -1.1784, -1.0406, -0.0736,
         0.2334,  0.4709,  0.2602,  0.8750,  1.2393,  0.8288,  0.7740])

In [12]:
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pack_sequence,pad_packed_sequence

def collate_fn(data):
    
    feature, label = [item[0] for item in data], [item[1] for item in data]
    feature.sort(key=lambda x: len(x), reverse=True)
    label.sort(key=lambda x: len(x), reverse=True)
    pad_feat = pad_sequence(feature, batch_first=True)    
    pad_label = pad_sequence(label, batch_first=True, padding_value=42)   
    packed_sample = pack_padded_sequence(pad_feat, [s.size(0) for s in feature] , batch_first=True)
    return packed_sample,pad_label

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [13]:
device

'cuda:0'

In [14]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Training

In [15]:
# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_dim=hidden_dim, hidden_layers=hidden_layer, isBidirectional=True, output_dim=41).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=42) 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [16]:
model

Classifier(
  (lstm): GRU(39, 35, num_layers=15, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Sequential(
    (0): Dropout(p=0.4, inplace=False)
    (1): Linear(in_features=70, out_features=41, bias=True)
  )
)

In [17]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features) 
        
        # reshape 
        outputs = outputs.view(-1,outputs.size(2))
        labels = labels.view(-1)
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)
                
                # reshape 
                outputs = outputs.view(-1,outputs.size(2))
                labels = labels.view(-1)
                
                loss = criterion(outputs, labels) 
                
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/train_total_len, train_loss/len(train_loader), val_acc/val_total_len, val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/val_total_len))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/train_total_len, train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


  0%|          | 0/39 [00:00<?, ?it/s]

[001/250] Train Acc: 0.207033 Loss: 3.141471 | Val Acc: 0.272952 loss: 2.676101
saving model with acc 0.273


  0%|          | 0/39 [00:00<?, ?it/s]

[002/250] Train Acc: 0.290940 Loss: 2.551694 | Val Acc: 0.335276 loss: 2.322670
saving model with acc 0.335


  0%|          | 0/39 [00:00<?, ?it/s]

...

[248/250] Train Acc: 0.836437 Loss: 0.553716 | Val Acc: 0.821178 loss: 0.649421


  0%|          | 0/39 [00:00<?, ?it/s]

[249/250] Train Acc: 0.836351 Loss: 0.554570 | Val Acc: 0.820051 loss: 0.655531


100%|██████████| 5/5 [00:01<00:00,  2.69it/s]

[250/250] Train Acc: 0.836387 Loss: 0.555315 | Val Acc: 0.819643 loss: 0.658627


## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [18]:
# load data train_tt_feat, train_tt_label, train_total_len
test_tt_feat, test_total_len = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_tt_feat, None)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)#, collate_fn=collate_fn)

0it [00:00, ?it/s]

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:10, 100.24it/s]

[INFO] test set


In [19]:
test_set[0]

tensor([[-0.9716, -0.0411, -0.0305,  ..., -0.9126, -1.4248, -0.9660],
        [-0.9313,  0.1897,  0.1694,  ..., -0.5289, -1.1123, -0.7526],
        [-0.9195,  0.3378,  0.2582,  ..., -0.0171, -0.3828,  0.1124],
        ...,
        [-1.3052, -0.1270,  0.4578,  ...,  0.2707, -0.0982, -0.0711],
        [-1.1118,  0.1894,  0.7075,  ...,  0.2134, -0.1544,  0.1309],
        [-1.2320, -0.0537,  0.0087,  ...,  0.1746, -0.1568, -0.0592]])

In [20]:
# x,y = iter(test_loader).next()
# print(y.shape)

In [21]:
# load model
modele = Classifier(input_dim=input_dim, hidden_dim=hidden_dim, hidden_layers=hidden_layer, isBidirectional=True, output_dim=41).to(device)
# criterion = nn.CrossEntropyLoss(ignore_index=42) 
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [22]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

ans = []
test_pred_list = []

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)
        
        # reshape 
        outputs = outputs.view(-1,outputs.size(2))
        ans.append(outputs)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        test_pred_list.append(test_pred)
        
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1078/1078 [03:03<00:00,  5.87it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [23]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))

## Hyper-parameters

In [24]:
# data prarameters
concat_nframes = 1              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.9               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 137986                   # random seed
batch_size = 100                # batch size
num_epoch = 250                 # the number of training epoch
learning_rate = 0.001           # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 #* concat_nframes # the input dim of the model, you should not change the value
hidden_layer = 15               # the number of hidden layers
hidden_dim = 35                # the hidden dim

## Prepare dataset and model

In [25]:
import gc

# preprocess data
train_tt_feat, train_tt_label, train_total_len = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_tt_feat, val_tt_label, val_total_len = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

520it [00:00, 5194.59it/s]

[Dataset] - # phone classes: 41, number of utterances for train: 3857


3857it [00:00, 5051.90it/s]
/tmp/ipykernel_3275/1127721884.py:105: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  print(np.array(tt_feat).shape,np.array(tt_label).shape)
/tmp/ipykernel_3275/1127721884.py:105: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(tt_feat).shape,np.array(tt_label).shape)


[INFO] train set
(3857,) (3857,)


429it [00:00, 4506.90it/s]

[Dataset] - # phone classes: 41, number of utterances for val: 429
[INFO] val set
(429,) (429,)


In [26]:
train_set = LibriDataset(train_tt_feat, train_tt_label)
val_set = LibriDataset(val_tt_feat, val_tt_label)

In [27]:
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pack_sequence,pad_packed_sequence

def collate_fn(data):
    
    feature, label = [item[0] for item in data], [item[1] for item in data]
    feature.sort(key=lambda x: len(x), reverse=True)
    label.sort(key=lambda x: len(x), reverse=True)
    pad_feat = pad_sequence(feature, batch_first=True)    
    pad_label = pad_sequence(label, batch_first=True, padding_value=42)   
    packed_sample = pack_padded_sequence(pad_feat, [s.size(0) for s in feature] , batch_first=True)
    return packed_sample,pad_label

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

## Training

In [28]:
# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_dim=hidden_dim, hidden_layers=hidden_layer, isBidirectional=True, output_dim=41).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=42) 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [29]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features) 
        
        # reshape 
        outputs = outputs.view(-1,outputs.size(2))
        labels = labels.view(-1)
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)
                
                # reshape 
                outputs = outputs.view(-1,outputs.size(2))
                labels = labels.view(-1)
                
                loss = criterion(outputs, labels) 
                
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/train_total_len, train_loss/len(train_loader), val_acc/val_total_len, val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/val_total_len))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/train_total_len, train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


  0%|          | 0/39 [00:00<?, ?it/s]

[001/250] Train Acc: 0.206662 Loss: 3.146137 | Val Acc: 0.291507 loss: 2.641462
saving model with acc 0.292


  0%|          | 0/39 [00:00<?, ?it/s]

[002/250] Train Acc: 0.298137 Loss: 2.547857 | Val Acc: 0.344370 loss: 2.336285
saving model with acc 0.344


  0%|          | 0/39 [00:00<?, ?it/s]

...

[248/250] Train Acc: 0.834644 Loss: 0.562880 | Val Acc: 0.815584 loss: 0.681876


  0%|          | 0/39 [00:00<?, ?it/s]

[249/250] Train Acc: 0.835281 Loss: 0.559398 | Val Acc: 0.814786 loss: 0.689104


100%|██████████| 5/5 [00:01<00:00,  2.72it/s]

[250/250] Train Acc: 0.835428 Loss: 0.559874 | Val Acc: 0.815327 loss: 0.685290


## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [30]:
# load data train_tt_feat, train_tt_label, train_total_len
test_tt_feat, test_total_len = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_tt_feat, None)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)#, collate_fn=collate_fn)

337it [00:00, 3360.61it/s]

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:00, 4511.36it/s]

[INFO] test set


In [31]:
# load model
modele = Classifier(input_dim=input_dim, hidden_dim=hidden_dim, hidden_layers=hidden_layer, isBidirectional=True, output_dim=41).to(device)
# criterion = nn.CrossEntropyLoss(ignore_index=42) 
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [32]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

ans1 = []
test_pred_list1 = []

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)
        
        # reshape 
        outputs = outputs.view(-1,outputs.size(2))
        ans1.append(outputs)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        test_pred_list1.append(test_pred)
        
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1078/1078 [03:07<00:00,  5.76it/s]


In [53]:
with open('prediction1.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))

In [57]:
pred.shape

(646268,)

# Ensemble

In [43]:
print(ans[0].shape)
print(len(ans))
print(ans1[0].shape)
print(len(ans1))

torch.Size([818, 41])
1078
torch.Size([818, 41])
1078


In [36]:
ans

[tensor([[ 6.6463e+00, -1.0226e-01,  4.5169e-01,  ..., -2.4879e-01,
          -1.0082e-02, -2.6639e+00],
         [ 8.4586e+00, -4.3862e-01,  3.8432e-01,  ..., -1.0065e-02,
          -3.1067e-01, -3.2597e+00],
         [ 9.7125e+00, -6.7473e-01,  3.1792e-01,  ...,  7.0497e-02,
          -4.6807e-01, -3.6606e+00],
         ...,
         [ 1.0782e+01, -1.0475e+00,  7.2254e-01,  ..., -8.1923e-01,
          -1.2347e+00, -3.1516e+00],
         [ 9.3838e+00, -9.6111e-01,  5.1676e-01,  ..., -8.4262e-01,
          -1.1402e+00, -2.4639e+00],
         [ 7.3304e+00, -7.1820e-01,  2.9433e-01,  ..., -8.0233e-01,
          -9.7628e-01, -1.6212e+00]], device='cuda:0'),
 tensor([[ 8.0362, -0.1874,  0.8883,  ...,  0.1333,  0.1411, -3.1962],
         [10.0322, -0.4987,  0.8840,  ...,  0.4275, -0.1114, -3.8707],
         [11.3247, -0.6749,  0.9159,  ...,  0.5533, -0.2195, -4.2888],
         ...,
         [10.8499, -1.3026,  0.4035,  ..., -0.5184, -0.7040, -3.3229],
         [ 9.3610, -1.1720,  0.2568,  .

In [37]:
ans1

[tensor([[ 7.0075,  1.2919, -0.6828,  ..., -1.6744,  0.4157, -3.2424],
         [ 8.8601,  0.9277, -0.4595,  ..., -1.2226,  0.0693, -3.8300],
         [10.2289,  0.6253, -0.3338,  ..., -0.8892, -0.2968, -4.2382],
         ...,
         [11.0846, -0.2417, -0.0257,  ...,  0.2822, -1.7138, -3.5499],
         [ 9.9132, -0.4903,  0.1937,  ...,  0.2914, -1.6667, -2.7553],
         [ 7.8629, -0.6742,  0.5326,  ...,  0.3486, -1.4487, -1.7264]],
        device='cuda:0'),
 tensor([[ 7.8091,  1.3507, -0.7307,  ..., -1.4627,  0.5629, -3.4110],
         [ 9.8554,  0.9766, -0.4890,  ..., -0.9402,  0.1893, -4.0123],
         [11.2943,  0.7112, -0.3418,  ..., -0.5418, -0.1695, -4.3872],
         ...,
         [11.7447, -0.5571, -0.2781,  ...,  0.1685, -1.6467, -3.7037],
         [10.4604, -0.7451,  0.0456,  ...,  0.1882, -1.6049, -2.9898],
         [ 8.3459, -0.8711,  0.4519,  ...,  0.2689, -1.4345, -1.9744]],
        device='cuda:0'),
 tensor([[ 7.0712,  1.3565, -0.3114,  ..., -1.3070,  0.5683, -3.28

In [50]:
((ans[0]+ans1[0])/2).shape

torch.Size([818, 41])

In [52]:
final = []
for i in range(len(ans)):
    final.append((ans[i]+ans1[i])/2)
print(len(final))

1078


In [54]:
pred_final = np.array([], dtype=np.int32)

for i in final:
    _, test_pred = torch.max(i, 1)
    pred_final = np.concatenate((pred_final, test_pred.cpu().numpy()), axis=0)

In [56]:
pred_final.shape

(646268,)

In [59]:
with open('prediction_final.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred_final):
        f.write('{},{}\n'.format(i, y))